In [311]:
import pandas as pd
from Bio.Seq import Seq
import copy

In [372]:
    def reversed_compl(seq):
        return str(Seq(seq).reverse_complement())

In [399]:
meta = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/kemai/20220517merTRflilter/mappingmanual/sc1/220417014_meta.csv')
meta.rename(columns={'Unnamed: 0':'barcode'},inplace=True)
meta['barcode'] = meta['barcode'].apply(lambda x: x.split('_')[1])
bg_barcode = set(meta[meta['celltype']=='TCells'].barcode)
bg_barcode = {reversed_compl(i) for i in bg_barcode}
len(bg_barcode)

347

In [400]:
df = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/kemai/20220512mergeTR/sc-1/03.assemble/assemble/sc-1_contig.csv', sep='\t', header=None)
df.columns = ['barcode', 'is_cell', 'contig_id', 'high_confidence', 'length', 'chain', 'v_gene', 'd_gene', 'j_gene', 'c_gene', 'full_length', 'productive', 'cdr3', 'cdr3_nt', 'reads', 'umis']
df['d_gene'] = df['d_gene'].apply(lambda x: x.split('(')[0] if not x == '*' else 'None')
df['c_gene'] = df['c_gene'].apply(lambda x: x.split('(')[0] if not x == '*' else 'None')
df['cdr3'] = df['cdr3_nt'].apply(lambda x: 'None' if "*" in str(Seq(x).translate()) or not len(x)%3==0 else str(Seq(x).translate()))
df['productive'] = df['cdr3'].apply(lambda x: False if x=='None' else True)

/Personal/chenjunjie/.local/lib/python3.7/site-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [401]:
def auto(df):
    n_cell_1_percentile = len(df) // 5
    sorted_counts = sorted(df.umis, reverse=True)
    count_cell_1_percentile = sorted_counts[n_cell_1_percentile]
    threshold = int(count_cell_1_percentile / 10)
    return threshold

In [402]:
df.sort_values(by='umis', ascending=False, inplace=True)
df_chain_heavy = df[df['chain']=='IGH']
df_chain_light = df[(df['chain']=='IGK') | (df['chain']=='IGL')]
df_chain_heavy = df_chain_heavy.drop_duplicates(['barcode'])
df_chain_light = df_chain_light.drop_duplicates(['barcode'])
df_for_clono = pd.concat([df_chain_heavy, df_chain_light], ignore_index=True)

In [403]:
cdr3_list = list(df_for_clono['cdr3'])
cdr3_list = [i for i in cdr3_list if i.startswith('C')]
cdr3_list = [i for i in cdr3_list if len(i)>=5]
cdr3_list = [i for i in cdr3_list if 'UAG' or 'UAA' or 'UGA' not in i]
df_for_clono = df_for_clono[df_for_clono['cdr3'].isin(cdr3_list)]

In [404]:
df_for_clono

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
0,TGATACGTTCAGGAGGTTGTTCCA,True,TGATACGTTCAGGAGGTTGTTCCA_899,True,517,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2B*03,True,True,CARGITGIDYW,TGTGCGAGGGGTATAACTGGAATTGACTACTGG,242751,105937
1,TAGACGGAGTCGCTATTCGAGCGT,True,TAGACGGAGTCGCTATTCGAGCGT_1286,True,605,IGH,IGHV3-23*04,IGHD7-27*01,IGHJ2*01,IGHG1*01,True,True,CAKDRNWGYEYFDLW,TGTGCGAAAGATCGAAACTGGGGATACGAGTACTTCGATCTCTGG,146639,79062
2,GTTGTCGGTCTCGGTTTGCATAGT,True,GTTGTCGGTCTCGGTTTGCATAGT_525,True,804,IGH,IGHV3-20*01,IGHD3-9*01,IGHJ4*02,IGHG2B*03,True,True,CARDLTLFDYW,TGTGCGAGAGATCTAACTCTTTTTGACTACTGG,138706,71297
3,TTCTGTGTGGTCGTGTCGATGTTT,True,TTCTGTGTGGTCGTGTCGATGTTT_5973,True,522,IGH,IGHV4-39*01,IGHD1-7*01,IGHJ2*01,IGHA*09,True,True,CASLYNWNYDWFDPWGLW,TGTGCGAGCCTGTATAACTGGAACTACGACTGGTTCGACCCCTGGG...,74544,45872
4,GTGTGTCGGCTAACTCTCGAAGTG,True,GTGTGTCGGCTAACTCTCGAAGTG_2191,True,536,IGH,IGHV3-48*03,IGHD3-22*01,IGHJ4*02,IGHA*09,True,True,CAREADWLVHYYFDYW,TGTGCGAGAGAGGCGGACTGGCTGGTTCATTACTACTTTGACTACTGG,55476,39029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,GACCTTAGGTCTTGGCGAGGATGG,True,GACCTTAGGTCTTGGCGAGGATGG_4323,True,499,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,32,28
12140,GCACTGTCTTCGCACCTACTTCGG,True,GCACTGTCTTCGCACCTACTTCGG_4316,True,566,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,30,27
12141,GGCTACAGTGGCTCAGGTGTCCTT,True,GGCTACAGTGGCTCAGGTGTCCTT_4312,True,520,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,34,26
12142,TCATTGAGGGATTAGGTGCGATCT,True,TCATTGAGGGATTAGGTGCGATCT_4308,True,510,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,26,22


In [405]:
        df_assign = copy.deepcopy(df_for_clono)
        df_assign.loc[~df_assign['barcode'].isin(bg_barcode),'umis'] = df_assign.loc[~df_assign['barcode'].isin(bg_barcode),'umis'].apply(lambda x:x*3)
        df_assign.loc[df_assign['barcode'].isin(bg_barcode),'umis'] = df_assign.loc[df_assign['barcode'].isin(bg_barcode),'umis'].apply(lambda x:x//3)

In [406]:
            df_chain_heavy = df_assign[df_assign['chain']=='IGH']
            df_chain_light = df_assign[(df_assign['chain']=='IGK') | (df_assign['chain']=='IGL')]
            threshold = auto(df_chain_heavy)
            df_chain_heavy = df_chain_heavy[df_chain_heavy['umis']>=threshold]
            threshold = auto(df_chain_light,)
            df_chain_light = df_chain_light[df_chain_light['umis']>=threshold]
            df_assign = pd.concat([df_chain_heavy, df_chain_light], ignore_index=True)

In [407]:
df_assign

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
0,TGATACGTTCAGGAGGTTGTTCCA,True,TGATACGTTCAGGAGGTTGTTCCA_899,True,517,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2B*03,True,True,CARGITGIDYW,TGTGCGAGGGGTATAACTGGAATTGACTACTGG,242751,317811
1,TAGACGGAGTCGCTATTCGAGCGT,True,TAGACGGAGTCGCTATTCGAGCGT_1286,True,605,IGH,IGHV3-23*04,IGHD7-27*01,IGHJ2*01,IGHG1*01,True,True,CAKDRNWGYEYFDLW,TGTGCGAAAGATCGAAACTGGGGATACGAGTACTTCGATCTCTGG,146639,237186
2,GTTGTCGGTCTCGGTTTGCATAGT,True,GTTGTCGGTCTCGGTTTGCATAGT_525,True,804,IGH,IGHV3-20*01,IGHD3-9*01,IGHJ4*02,IGHG2B*03,True,True,CARDLTLFDYW,TGTGCGAGAGATCTAACTCTTTTTGACTACTGG,138706,213891
3,TTCTGTGTGGTCGTGTCGATGTTT,True,TTCTGTGTGGTCGTGTCGATGTTT_5973,True,522,IGH,IGHV4-39*01,IGHD1-7*01,IGHJ2*01,IGHA*09,True,True,CASLYNWNYDWFDPWGLW,TGTGCGAGCCTGTATAACTGGAACTACGACTGGTTCGACCCCTGGG...,74544,137616
4,GTGTGTCGGCTAACTCTCGAAGTG,True,GTGTGTCGGCTAACTCTCGAAGTG_2191,True,536,IGH,IGHV3-48*03,IGHD3-22*01,IGHJ4*02,IGHA*09,True,True,CAREADWLVHYYFDYW,TGTGCGAGAGAGGCGGACTGGCTGGTTCATTACTACTTTGACTACTGG,55476,117087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,TGCGTGAATAAGCGTTTAGAACAC,True,TGCGTGAATAAGCGTTTAGAACAC_2057,True,527,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,392,891
7708,TCTCACGGGAGGATGGGGTGAGTT,True,TCTCACGGGAGGATGGGGTGAGTT_1982,True,543,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,351,891
7709,TCATCCTAGGTCGTGTCGATGTTT,True,TCATCCTAGGTCGTGTCGATGTTT_1967,True,524,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,357,888
7710,GAGCCAATGGTGAGTTTCCAGTCG,True,GAGCCAATGGTGAGTTTCCAGTCG_502,True,524,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,347,885


In [408]:
df_for_clono = df_for_clono[df_for_clono['contig_id'].isin(set(df_assign['contig_id']))]

In [409]:
df_for_clono

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
0,TGATACGTTCAGGAGGTTGTTCCA,True,TGATACGTTCAGGAGGTTGTTCCA_899,True,517,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2B*03,True,True,CARGITGIDYW,TGTGCGAGGGGTATAACTGGAATTGACTACTGG,242751,105937
1,TAGACGGAGTCGCTATTCGAGCGT,True,TAGACGGAGTCGCTATTCGAGCGT_1286,True,605,IGH,IGHV3-23*04,IGHD7-27*01,IGHJ2*01,IGHG1*01,True,True,CAKDRNWGYEYFDLW,TGTGCGAAAGATCGAAACTGGGGATACGAGTACTTCGATCTCTGG,146639,79062
2,GTTGTCGGTCTCGGTTTGCATAGT,True,GTTGTCGGTCTCGGTTTGCATAGT_525,True,804,IGH,IGHV3-20*01,IGHD3-9*01,IGHJ4*02,IGHG2B*03,True,True,CARDLTLFDYW,TGTGCGAGAGATCTAACTCTTTTTGACTACTGG,138706,71297
3,TTCTGTGTGGTCGTGTCGATGTTT,True,TTCTGTGTGGTCGTGTCGATGTTT_5973,True,522,IGH,IGHV4-39*01,IGHD1-7*01,IGHJ2*01,IGHA*09,True,True,CASLYNWNYDWFDPWGLW,TGTGCGAGCCTGTATAACTGGAACTACGACTGGTTCGACCCCTGGG...,74544,45872
4,GTGTGTCGGCTAACTCTCGAAGTG,True,GTGTGTCGGCTAACTCTCGAAGTG_2191,True,536,IGH,IGHV3-48*03,IGHD3-22*01,IGHJ4*02,IGHA*09,True,True,CAREADWLVHYYFDYW,TGTGCGAGAGAGGCGGACTGGCTGGTTCATTACTACTTTGACTACTGG,55476,39029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847,TGCGTGAATAAGCGTTTAGAACAC,True,TGCGTGAATAAGCGTTTAGAACAC_2057,True,527,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,392,297
9848,TCTCACGGGAGGATGGGGTGAGTT,True,TCTCACGGGAGGATGGGGTGAGTT_1982,True,543,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,351,297
9849,TCATCCTAGGTCGTGTCGATGTTT,True,TCATCCTAGGTCGTGTCGATGTTT_1967,True,524,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,357,296
9850,GAGCCAATGGTGAGTTTCCAGTCG,True,GAGCCAATGGTGAGTTTCCAGTCG_502,True,524,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,347,295


In [410]:
df_for_clono_pro = df_for_clono[df_for_clono['productive']==True]
cell_barcodes = set(df_for_clono_pro['barcode'].tolist())
total_cells =len(cell_barcodes)

In [411]:
total_cells

4157

In [412]:
len(set(df_for_clono_pro.barcode).intersection(bg_barcode))

27

In [309]:
def get_vj_annot(df, chains, pairs):
    fl_pro_pair_df = pd.DataFrame(df[df['productive']==True].barcode.value_counts())
    fl_pro_pair_df = fl_pro_pair_df[fl_pro_pair_df['barcode']>=2]
    Result = []
    cell_nums = len(set(df['barcode'].tolist()))
    Result.append({
        'name': 'Cells With Productive V-J Spanning Pair',
        'value': fl_pro_pair_df.shape[0],
        'total': cell_nums,
    })
    for p in pairs:
        chain1 = p.split('_')[0]
        chain2 = p.split('_')[1]
        cbs1 = set(df[(df['full_length']==True)&(df['productive']==True)&(df['chain']==chain1)].barcode.tolist())
        cbs2 = set(df[(df['full_length']==True)&(df['productive']==True)&(df['chain']==chain2)].barcode.tolist())
        paired_cbs = len(cbs1.intersection(cbs2))
        Result.append({
            'name': f'Cells With Productive V-J Spanning ({chain1}, {chain2}) Pair',
            'value': paired_cbs,
            'total': cell_nums,
            'help_info': "Fraction of cell-associated barcodes with one productive contig for each chain of the receptor pair.A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start codon was found in the expected part of the V sequence, an in-frame CDR3 amino acid motif was found, and no stop codons were found in the aligned V-J region"
        })
    for c in chains:
        Result.append({
            'name': f'Cells With {c} Contig',
            'value': len(set(df[df['chain']==c].barcode.tolist())),
            'total': cell_nums,
            'help_info': f'Fraction of cell-associated barcodes with at least one {c} contig annotated as a full or partial V(D)J gene'
        })
        Result.append({
            'name': f'Cells With CDR3-annotated {c} Contig',
            'value': len(set(df[(df['chain']==c)&(df['productive']==True)].barcode.tolist())),
            'total': cell_nums,
        })
        Result.append({
            'name': f'Cells With V-J Spanning {c} Contig',
            'value': len(set(df[(df['full_length']==True)&(df['chain']==c)].barcode.tolist())),
            'total': cell_nums,
            'help_info': f"Fraction of cell-associated barcodes with at least one contig spanning the 5' end of the V region to the 3' end of the J region for {c}"
        })
        Result.append({
            'name': f'Cells With Productive {c} Contig',
            'value': len(set(df[(df['full_length']==True)&(df['productive']==True)&(df['chain']==c)].barcode.tolist())),
            'total': cell_nums,
            'help_info': "Fraction of cell-associated barcodes with productive IGL chain. A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start codon was found in the expected part of the V sequence, an in-frame CDR3 amino acid motif was found, and no stop codons were found in the aligned V-J region"
        })

    return Result

In [310]:
get_vj_annot(df_for_clono, ['IGH', 'IGL', 'IGK'], ['IGH_IGL', 'IGH_IGK'])

[{'name': 'Cells With Productive V-J Spanning Pair',
  'value': 3555,
  'total': 4157},
 {'name': 'Cells With Productive V-J Spanning (IGH, IGL) Pair',
  'value': 0,
  'total': 4157,
  'help_info': "Fraction of cell-associated barcodes with one productive contig for each chain of the receptor pair.A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start codon was found in the expected part of the V sequence, an in-frame CDR3 amino acid motif was found, and no stop codons were found in the aligned V-J region"},
 {'name': 'Cells With Productive V-J Spanning (IGH, IGK) Pair',
  'value': 3555,
  'total': 4157,
  'help_info': "Fraction of cell-associated barcodes with one productive contig for each chain of the receptor pair.A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start c